In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
201,AFG,Asia,Afghanistan,2020-07-29,36471.0,103.0,1271.0,1.0,936.875,2.646,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
344,ALB,Europe,Albania,2020-07-29,4997.0,117.0,148.0,4.0,1736.396,40.656,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
551,DZA,Africa,Algeria,2020-07-29,27973.0,0.0,1163.0,0.0,637.910,0.000,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-28,France,183252.0,0.0,30226.0,0.0,0.0,0.0
0,2020-07-28,China,86947.0,0.0,4664.0,0.0,0.0,0.0
0,2020-07-28,Italy,246655.0,0.0,35140.0,0.0,0.0,0.0
0,2020-07-28,Spain,278766.0,0.0,28445.0,0.0,0.0,0.0
0,2020-07-28,United States,4364123.0,0.0,148677.0,0.0,0.0,0.0
0,2020-07-28,World,16860694.0,0.0,660315.0,0.0,0.0,0.0
0,2020-07-28,United Kingdom,301097.0,0.0,45834.0,0.0,0.0,0.0
0,2020-07-28,Germany,206651.0,0.0,9131.0,0.0,0.0,0.0
0,2020-07-28,Iran,297147.0,0.0,16111.0,0.0,0.0,0.0
0,2020-07-28,Turkey,227903.0,0.0,5646.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-29,France,183252.0,183804.0,30226.0,30223.0,-552.0,3.0
0,2020-07-29,China,86947.0,86990.0,4664.0,4657.0,-43.0,7.0
0,2020-07-29,Italy,246655.0,246488.0,35140.0,35123.0,167.0,17.0
0,2020-07-28,Spain,278766.0,280610.0,28445.0,28436.0,-1844.0,9.0
0,2020-07-29,United States,4364123.0,4351997.0,148677.0,149256.0,12126.0,-579.0
0,2020-07-29,World,16860694.0,16708920.0,660315.0,660123.0,151774.0,192.0
0,2020-07-29,United Kingdom,301097.0,300692.0,45834.0,45878.0,405.0,-44.0
0,2020-07-29,Germany,206651.0,206926.0,9131.0,9128.0,-275.0,3.0
0,2020-07-29,Iran,297147.0,296273.0,16111.0,16147.0,874.0,-36.0
0,2020-07-29,Turkey,227903.0,227982.0,5646.0,5645.0,-79.0,1.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")